In [2]:
import hf00
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from IPython.display import display, Audio
import os
import torchaudio.transforms as T
import librosa
import pandas as pd
import torch

In [3]:
aud_sample = '../../LibriVox_Kaggle/achtgesichterambiwasse/achtgesichterambiwasse_0050.wav'
spectro_fn = T.Spectrogram(power=None)
inv_spectro_fn = T.InverseSpectrogram()

audio, label = hf00.get_random_audio_sec(aud_sample)

aud_spec = spectro_fn(audio)
label_spec = spectro_fn(label)
aud_new = inv_spectro_fn(aud_spec)

In [4]:
aud_spec.shape, label_spec.shape

(torch.Size([1, 201, 81]), torch.Size([1, 201, 81]))

In [5]:
class audioDataset(Dataset):

    def __init__(self, audio_df, audio_dir):
        #self.audio_df = pd.read_csv(audio_csvfile)
        self.audio_df = audio_df
        self.audio_dir = audio_dir

    def __len__(self):
        return len(self.audio_df)
    
    def __getitem__(self, index):
        audio_path = os.path.join(self.audio_dir, self.audio_df.iloc[index, 0])

        audio, label = hf00.get_random_audio_sec(audio_path)

        audio_spec = spectro_fn(audio)
        label_spec = spectro_fn(label)

        return audio_spec.real, label_spec.real

In [6]:
aud_dir = '../../LibriVox_Kaggle/'
train_df = pd.read_csv('only_audioFname_train.csv')
test_df = pd.read_csv('only_audioFname_test.csv')

train_df = train_df[0:2000]
test_df = test_df[0:500]

train_dataset = audioDataset(train_df,audio_dir=aud_dir)
test_dataset = audioDataset(test_df,audio_dir=aud_dir)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [25]:
x = torch.randn(aud_spec.unsqueeze(0).shape)

enc = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
    nn.LeakyReLU(),
    nn.Conv2d(64, 32, kernel_size=3, stride=2,padding=1),
    nn.LeakyReLU(),
    nn.Conv2d(32, 16, kernel_size=3, stride=2,padding=1),
    nn.LeakyReLU(),
    nn.Conv2d(16, 8, kernel_size=3, stride=1,padding=1),
    #nn.LeakyReLU()
    )

dec = nn.Sequential(
    nn.ConvTranspose2d(8, 16, kernel_size=3, stride=1, padding=1),
    nn.LeakyReLU(),
    nn.ConvTranspose2d(16, 32, kernel_size=3, stride=2,padding=1),
    nn.LeakyReLU(),
    nn.ConvTranspose2d(32, 64, kernel_size=3, stride=2,padding=1),
    nn.LeakyReLU(),
    nn.ConvTranspose2d(64, 1, kernel_size=3, stride=1,padding=1),
    nn.LeakyReLU(),
    
    nn.Sigmoid()
)

enc_x = enc(x)
x.shape, enc_x.shape, dec(enc_x).shape

(torch.Size([1, 1, 201, 81]),
 torch.Size([1, 8, 51, 21]),
 torch.Size([1, 1, 201, 81]))